In [1]:
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import os
from sklearn import preprocessing
import tensorflow as tf

In [5]:
tf.compat.v2.test.is_gpu_available()

False

In [35]:
df = pd.read_csv('../data/annotations.csv')
df

,id,filename,width,height,depth,name,xmin,xmax,ymin,ymax
0,0,n02085620_10074,333,500,3,Chihuahua,25,276,10,498
1,1,n02085620_10131,395,495,3,Chihuahua,49,393,9,493
2,2,n02085620_10621,500,298,3,Chihuahua,142,335,43,250
3,3,n02085620_1073,345,500,3,Chihuahua,0,312,27,498
4,4,n02085620_10976,322,484,3,Chihuahua,90,242,104,452
...,...,...,...,...,...,...,...,...,...,...
22121,22121,n02116738_9818,500,333,3,African_hunting_dog,48,499,0,332
22122,22122,n02116738_9829,202,300,3,African_hunting_dog,0,170,39,299
22123,22123,n02116738_9844,600,400,3,African_hunting_dog,199,473,136,348
22124,22124,n02116738_9844,600,400,3,African_hunting_dog,235,539,140,334


In [36]:
df = df.loc[(df["name"] == 'Chihuahua') | (df["name"] == 'African_hunting_dog')]
df


,id,filename,width,height,depth,name,xmin,xmax,ymin,ymax
0,0,n02085620_10074,333,500,3,Chihuahua,25,276,10,498
1,1,n02085620_10131,395,495,3,Chihuahua,49,393,9,493
2,2,n02085620_10621,500,298,3,Chihuahua,142,335,43,250
3,3,n02085620_1073,345,500,3,Chihuahua,0,312,27,498
4,4,n02085620_10976,322,484,3,Chihuahua,90,242,104,452
...,...,...,...,...,...,...,...,...,...,...
22121,22121,n02116738_9818,500,333,3,African_hunting_dog,48,499,0,332
22122,22122,n02116738_9829,202,300,3,African_hunting_dog,0,170,39,299
22123,22123,n02116738_9844,600,400,3,African_hunting_dog,199,473,136,348
22124,22124,n02116738_9844,600,400,3,African_hunting_dog,235,539,140,334


In [37]:
le = sklearn.preprocessing.LabelEncoder()
labels = le.fit_transform(df["name"])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
ids = df["id"].astype(int).to_numpy()

In [47]:
class DataLoader(keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.shuffle()
        
    def shuffle(self):
            indeces = np.random.permutation(x.shape[0])
            return x[indeces], y[indeces]
        
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array([
            np.array(np.load("../data/processed/" + str(file_name)+ ".npy")) for file_name in batch_x]), np.array(batch_y)

In [48]:
dg = DataGenerator(ids, labels, 32)


In [49]:
x, y  = dg.__getitem__(0)

In [50]:
y.shape

(32,)

In [51]:
x.shape

(32, 128, 128, 3)

In [52]:
x[0].shape

(128, 128, 3)

In [53]:

import tensorflow.keras as keras
from tensorflow.keras.layers import concatenate, Conv2D, Dense, Flatten, Layer, MaxPooling2D
class Inception(Layer):

    """
    This is an example of how can you define your own layer from existing layers. It is very similar to how we define
    models.

    Change the __init__ and call methods so that the Inception layer looks like the layer depicted in the figure
    in the notebook. The Inception layer consists of four branches, that are concatenated at the end.

    You should be able to build this layer from the layers you have already seen - Conv2D, MaxPooling2D and concatenate.

    Several notes:
        1) The 3x3 max-pooling layer in the 4th branch has 1x1 stride
        2) You should use concatenate with lowercase `c` at the start. Concatenate with uppercase `C` is slightly
        different and does not work properly in Layer definitions.
    """

    def __init__(self, filters, activation):
        """
        :param filters: How many filters are in the convolutional layers within this layer.
        :param activation: What is the activation function used
        """
        super(Inception, self).__init__()

        # FIXME: Initialize all the layers you need for the Inception layer
        self.conv_1_1 = Conv2D(
            filters=filters,
            kernel_size=1,
            padding='same',
            activation=activation)
        self.conv_2_1 = Conv2D(
            filters=filters,
            kernel_size=1,
            padding='same',
            activation=activation)
        self.conv_2_2 = Conv2D(
            filters=filters,
            kernel_size=3,
            padding='same',
            activation=activation)
        self.conv_3_1 = Conv2D(
            filters=filters,
            kernel_size=1,
            padding='same',
            activation=activation)
        self.conv_3_2 = Conv2D(
            filters=filters,
            kernel_size=5,
            padding='same',
            activation=activation)
        self.maxpool_4_1 = MaxPooling2D(
            pool_size=3,
            strides=1,
            padding='same')
        self.conv_4_2 = Conv2D(
            filters=filters,
            kernel_size=1,
            padding='same',
            activation=activation)


    def call(self, x):
        # FIXME: Build the Inception layer
        x1 = self.conv_1_1(x)
        x2 = self.conv_2_2(self.conv_2_1(x))
        x3 = self.conv_3_2(self.conv_3_1(x))
        x4 = self.conv_4_2(self.maxpool_4_1(x))
        x = concatenate([x1, x2, x3, x4], axis=3)
        return x




In [54]:
class InceptionNet(keras.Model):
    """
    Inception version of the simple CNN we used previously. You do not need to change anything here.
    """

    def __init__(self, filters, dim_output):
        super(InceptionNet, self).__init__()
        self.model_layers = [
            Inception(
                filters=filters,
                activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Inception(
                filters=filters,
                activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Inception(
                filters=filters,
                activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Inception(
                filters=filters,
                activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Flatten(),
            Dense(
                units=128,
                activation='relu'),
            Dense(
                units=dim_output,
                activation='softmax')
        ]

    def call(self, x):
        for layer in self.model_layers:
            x = layer(x)
        return x


In [57]:
model = InceptionNet(32, dim_output=2)
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
opt = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)


In [ ]:
model.fit_generator(dg, steps_per_epoch=None, epochs=30, verbose=1, callbacks=None, validation_data=None, validation_steps=None, validation_freq=1, class_weight=None, max_queue_size=10, workers=6, use_multiprocessing=False, shuffle=True, initial_epoch=0)


Epoch 1/30
12/12 [==============================] - 45s 4s/step - loss: 0.6604 - accuracy: 0.6819
Epoch 2/30
12/12 [==============================] - 44s 4s/step - loss: 0.5991 - accuracy: 0.7170
Epoch 3/30
12/12 [==============================] - 44s 4s/step - loss: 0.5937 - accuracy: 0.7493
Epoch 4/30
12/12 [==============================] - 49s 4s/step - loss: 0.5327 - accuracy: 0.7682
Epoch 5/30
12/12 [==============================] - 50s 4s/step - loss: 0.5373 - accuracy: 0.7466
Epoch 6/30
12/12 [==============================] - 47s 4s/step - loss: 0.5943 - accuracy: 0.7278
Epoch 7/30
12/12 [==============================] - 50s 4s/step - loss: 0.4836 - accuracy: 0.7790
Epoch 8/30
12/12 [==============================] - 49s 4s/step - loss: 0.5698 - accuracy: 0.7385
Epoch 9/30
12/12 [==============================] - 48s 4s/step - loss: 0.4475 - accuracy: 0.8059
Epoch 10/30
12/12 [==============================] - 50s 4s/step - loss: 0.4768 - accuracy: 0.7951
Epoch 11/30
12/12 [

In [ ]:
model.summray()

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()


In [ ]:
train_labels[0]